In [28]:
import requests
import pandas as pd
import numpy as np
import math
import json

In [29]:
df = pd.read_csv('data/demo_intermediate.csv', index_col=False)

In [30]:
df.head()

,B01001_001E,B01001_001M,B01001_002E,B01001_002M,B01001_003E,B01001_003M,B01001_004E,B01001_004M,B01001_005E,B01001_005M,...,DP05_0088E,DP05_0088M,DP05_0088PE,DP05_0088PM,DP05_0089E,DP05_0089M,DP05_0089PE,DP05_0089PM,GEO_ID,NAME
0,32127.0,1246.122386,14961.0,712.721545,694.0,178.555314,937.0,224.926655,1020.0,250.339769,...,9935.0,578.784934,NaN,NaN,11968.0,652.078983,NaN,NaN,BX31,BX31
1,57718.0,1775.077463,27570.0,1175.599422,2768.0,516.036820,1932.0,331.407604,1993.0,323.620148,...,13717.0,916.604604,NaN,NaN,16104.0,830.780958,NaN,NaN,BX05,BX05
2,29990.0,1432.393452,13655.0,964.579701,869.0,237.491052,1232.0,321.619340,1003.0,315.680218,...,7037.0,596.124148,NaN,NaN,8776.0,631.763405,NaN,NaN,BX06,BX06
3,38083.0,1482.243570,18710.0,1032.483898,1660.0,380.988189,1240.0,297.465964,1169.0,288.263768,...,10555.0,741.169346,NaN,NaN,12806.0,706.343401,NaN,NaN,BX07,BX07
4,35856.0,1580.290163,15986.0,1231.871341,1703.0,411.582313,1559.0,310.049996,1327.0,271.239746,...,8390.0,1023.547263,NaN,NaN,11047.0,687.457635,NaN,NaN,BX01,BX01


### Formula:
https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf

In [33]:
# e --> estimate, m --> moe
def get_c(e, m): 
    if str(m).strip():
        return ''
    elif e == 0:
        return ''
    else: 
        return m/1.645/e*100

def get_p(e, agg_e):
    if agg_e == 0: 
        return ''
    else: 
        return e/agg_e*100

def get_z(e, m, p, agg_e, agg_m):
    if p == 0:
        return ''
    elif p == 100:
        return ''
    elif agg_e == 0:
        return ''
    elif m**2 - (e*agg_m/agg_e)**2 <0:
        return math.sqrt(m**2 + (e*agg_m/agg_e)**2)/agg_e*100
    else: 
        return math.sqrt(m**2 - (e*agg_m/agg_e)**2)/agg_e*100

In [35]:
def find_total(variable, stat='E'):
    if variable[0] == 'B' or variable[0] == 'C' : 
        return f"{variable.split('_')[0]}_001{stat}"
    elif variable[0] == 'D': 
        return f"{variable.split('_')[0]}_0001{stat}"
    else: #S1810_C01_001M
        return f"{'_'.join(variable.split('_')[:2])}_001{stat}"

In [36]:
%%time
variables = list(df.columns)
variables.remove('GEO_ID')
variables.remove('NAME')
var = list(set([i[:-1] for i in variables if i[:-1][-1] != 'P']))

def find_total(variable, stat='E'):
    if variable[0] == 'B' or variable[0] == 'C' : 
        return f"{variable.split('_')[0]}_001{stat}"
    elif variable[0] == 'D': 
        return f"{variable.split('_')[0]}_0001{stat}"
    else: #S1810_C01_001M
        return f"{'_'.join(variable.split('_')[:2])}_001{stat}"

for i in var:
    df.loc[:,f'{i}C'] = df.apply(lambda row: get_c(row[f'{i}E'], 
                                                       row[f'{i}M']), axis=1)
    
    if int(i.split('_')[-1]) == 1:
        df.loc[:,f'{i}Z'] = ''
        df.loc[:,f'{i}P'] = 100
    else:
        total_e = find_total(i, 'E')
        total_m = find_total(i, 'M')
        df.loc[:,f'{i}P'] = df.apply(lambda row: get_p(row[f'{i}E'], 
                                                       row[total_e]), axis=1)
        
        df.loc[:,f'{i}Z'] = df.apply(lambda row: get_z(row[f'{i}E'], 
                                                       row[f'{i}M'], 
                                                       row[f'{i}P'], 
                                                       row[total_e], 
                                                       row[total_m]), axis=1)
        # for i in var:
#     df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}C'] = df.apply(lambda row: get_c(row[f'{i}E'], row[f'{i}M']), axis=1)
    
#     if f'{i}PE' in list(df.columns):
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}PE']
#     else: 
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.apply(lambda row: get_p(row[f'{i}E'], 
#                                                                                                 city_level[f'{i}E']), axis=1)
#     if f'{i}PM' in list(df.columns):
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}PM']
#     else: 
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}Z'] = df.apply(lambda row: get_z(row[f'{i}E'], 
#                                                                                                 row[f'{i}M'], 
#                                                                                                 row[f'{i}P'], 
#                                                                                                 city_level[f'{i}E'], 
#                                                                                                 city_level[f'{i}M']), axis=1)

CPU times: user 7min 17s, sys: 21.8 s, total: 7min 39s
Wall time: 7min 45s


In [43]:
output_cols = [[i+'E', i+'M', i+'P', i+'Z', i+'C'] for i in var]

In [44]:
output = ['GEO_ID', 'NAME']
for i in output_cols:
    output+=i

In [45]:
df[output].head()

,GEO_ID,NAME,DP05_0082E,DP05_0082M,DP05_0082P,DP05_0082Z,DP05_0082C,B01001_044E,B01001_044M,B01001_044P,...,B03001_014E,B03001_014M,B03001_014P,B03001_014Z,B03001_014C,B03001_024E,B03001_024M,B03001_024P,B03001_024Z,B03001_024C
0,BX31,BX31,395.0,245.181565,1.2295,0.761672,,361.0,146.372812,1.12367,...,61.0,70.313583,0.189871,0.218737,,4.0,37.161808,0.0124506,0.115671,
1,BX05,BX05,243.0,156.435290,0.421013,0.270724,,467.0,164.109719,0.809106,...,585.0,342.563863,1.01355,0.592694,,0.0,46.270941,0,,
2,BX06,BX06,96.0,93.413061,0.320107,0.311105,,117.0,84.917607,0.39013,...,423.0,331.273301,1.41047,1.10256,,0.0,31.780497,0,,
3,BX07,BX07,350.0,250.037997,0.919045,0.655586,,473.0,156.553505,1.24202,...,266.0,204.014705,0.698474,0.53502,,0.0,35.185224,0,,
4,BX01,BX01,114.0,130.820488,0.317938,0.36458,,439.0,305.198296,1.22434,...,106.0,164.432357,0.295627,0.458406,,0.0,37.054015,0,,


In [40]:
df[['B01001_001E', 'B01001_001M', 'B01001_001C', 'B01001_001P', 'B01001_001Z']].head()

,B01001_001E,B01001_001M,B01001_001C,B01001_001P,B01001_001Z
0,32127.0,1246.122386,,100,NaN
1,57718.0,1775.077463,,100,NaN
2,29990.0,1432.393452,,100,NaN
3,38083.0,1482.243570,,100,NaN
4,35856.0,1580.290163,,100,NaN


#### Notes: 
for this following group, we do have e.g. Percent Margin of Error, maybe we shouldn't calculate these
```json
"DP05_0089PM": {
    "label": "Percent Margin of Error!!CITIZEN VOTING AGE POPULATION!!Citizen 18 and over population!!Female",
    "concept": "ACS DEMOGRAPHIC AND HOUSING ESTIMATES",
    "predicateType": "float",
    "group": "DP05",
    "limit": 0,
    "predicateOnly": true
},
```
https://api.census.gov/data/2017/acs/acs5/profile/groups/DP05.json

In [46]:
df[output].to_csv('data/demo_final.csv', index=False)